## Dataset Generation

In [5]:
import numpy as np
import pandas as pd

# -----------------------------
# Setup
# -----------------------------
rng = np.random.default_rng(42)
N = 1000                   # ≥500 rows
TARGET_PREV = 0.35         # ≥30% cancer cases

# -----------------------------
# Sampling helpers (true truncation)
# -----------------------------
def rtruncnorm(mean, sd, low, high, size):
    """True truncated Normal via rejection sampling (no piling at bounds)."""
    out = np.empty(size)
    filled = 0
    while filled < size:
        need = size - filled
        draws = rng.normal(mean, sd, need * 2)
        keep = draws[(draws >= low) & (draws <= high)]
        k = min(keep.size, need)
        out[filled:filled+k] = keep[:k]
        filled += k
    return out

def rtrunclognorm(median, sigma, low, high, size):
    """True truncated Log-normal via rejection on the linear scale."""
    mu = np.log(median)
    out = np.empty(size)
    filled = 0
    while filled < size:
        need = size - filled
        draws = rng.lognormal(mean=mu, sigma=sigma, size=need*2)
        keep = draws[(draws >= low) & (draws <= high)]
        k = min(keep.size, need)
        out[filled:filled+k] = keep[:k]
        filled += k
    return out

def sigmoid(x):
    return 1/(1+np.exp(-x))

# -----------------------------
# 1) Baseline demographics (unconditional)
#    Diagnosis will be derived from risk later.
# -----------------------------

# Age: broad population; class means will emerge via risk (older → higher risk)
age = rtruncnorm(mean=50, sd=12, low=18, high=90, size=N)

# Menopause: age-conditioned; steepened so P(post|55)≈0.85
p_post = sigmoid((age - 51)/2.3)
menopausal_status = (rng.random(N) < p_post).astype(int)

# BMI: population-level (class means will emerge)
bmi = rtruncnorm(mean=26.8, sd=5.1, low=16, high=45, size=N)

# Parity: population draw; protective effect will be applied in the risk model
# (Poisson-like, then truncate to [0,8])
parity = np.clip(rng.poisson(lam=1.8, size=N), 0, 8).astype(int)

# Family history (FH): base prevalence; risk model will enrich among cases
family_history = (rng.random(N) < 0.12).astype(int)  # ~midpoint between 0.08 and 0.22

# MHT use: only among post-menopause; ~0 in pre-menopause
mht_use = np.zeros(N, dtype=int)
post_idx = menopausal_status == 1
mht_use[post_idx] = (rng.random(post_idx.sum()) < 0.22).astype(int)  # ~20–25% post-meno

# BRCA status: rare in population, higher if FH=1; risk model will enrich among cases
brca_base = np.where(family_history==1, 0.02, 0.003)  # small bump with FH
brca_status = (rng.random(N) < brca_base).astype(int)

# -----------------------------
# 2) Latent cancer risk (logistic) with specified interactions
#    - BMI→risk = 0 if MHT=1 or FH=1, else weak + slope (OR≈1.3–1.6 obese vs normal)
#    - Parity protective (OR≈0.87 per birth)
#    - BRCA & FH strong main effects; Age raises risk
#    - MHT modest + effect (OR≈1.3)
# -----------------------------
# BMI term active only if MHT=0 and FH=0 (effect-modification)
z_bmi = (bmi - 25)/5.0
bmi_effect = np.zeros(N)
active_bmi = (mht_use == 0) & (family_history == 0)
bmi_effect[active_bmi] = np.log(1.45) * z_bmi[active_bmi]  # weak + slope

# Linear predictor excluding intercept
lp_wo_b = (
    np.log(1.8) * ((age - 50)/10)   # age per decade (older → higher risk)
  + np.log(8.0) * brca_status       # BRCA big effect
  + np.log(2.0) * family_history    # FH increased risk
  + np.log(1.3) * mht_use           # MHT modest + effect
  + np.log(0.87) * parity           # protective per birth
  + bmi_effect                       # BMI effect if not nullified
)

# Calibrate intercept to hit target prevalence (~35%)
def calibrate_intercept(lp_x, target, lo=-8.0, hi=8.0, iters=40):
    for _ in range(iters):
        mid = 0.5*(lo+hi)
        p = sigmoid(mid + lp_x).mean()
        if p < target:
            lo = mid
        else:
            hi = mid
    return 0.5*(lo+hi)

b0 = calibrate_intercept(lp_wo_b, TARGET_PREV)
p_cancer = sigmoid(b0 + lp_wo_b)

# Draw diagnosis from calibrated risk
diagnosis = (rng.random(N) < p_cancer).astype(int)
ctrl_idx = (diagnosis == 0)
case_idx = (diagnosis == 1)

# -----------------------------
# 3) Imaging (tumor size) conditional on diagnosis
# -----------------------------
tumor_size_cm = np.zeros(N, dtype=float)

# Controls: zero-inflated (no lesion) + benign simple cysts 1–5 cm
ctrl_ids = np.where(ctrl_idx)[0]
ctrl_has_cyst = rng.random(ctrl_ids.size) < 0.30  # ~70% zeros
cyst_ids = ctrl_ids[ctrl_has_cyst]
tumor_size_cm[cyst_ids] = rtruncnorm(mean=3.0, sd=1.0, low=1.0, high=5.0, size=cyst_ids.size)

# Cases: right-skewed sizes, median ~9.5 cm, clip to 0.5–20
case_ids = np.where(case_idx)[0]
case_sizes = rtrunclognorm(median=9.5, sigma=0.5, low=0.5, high=20.0, size=case_ids.size)
tumor_size_cm[case_ids] = case_sizes

# -----------------------------
# 4) CA-125 conditional on diagnosis (with healthy-only hemodilution)
# -----------------------------
ca125 = np.zeros(N, dtype=float)

# Healthy baseline:
#   - Post-meno median ~14
#   - Pre-meno median ~18 (physiology & benign overlap → slightly higher)
#   - Log-normal spread 0.50–0.55; false-positive tail 1–3% >35
pre_ctrl = ctrl_idx & (menopausal_status == 0)
post_ctrl = ctrl_idx & (menopausal_status == 1)

ca125[post_ctrl] = rtrunclognorm(median=14.0, sigma=0.50, low=2, high=2000, size=post_ctrl.sum())
ca125[pre_ctrl]  = rtrunclognorm(median=18.0, sigma=0.55, low=2, high=2000, size=pre_ctrl.sum())

# BMI hemodilution (healthy only): −2% per BMI unit from 25, cap ±20%
delta_bmi_ctrl = bmi[ctrl_idx] - 25.0
hemo = np.clip(np.exp(delta_bmi_ctrl * np.log(0.98)), 0.80, 1.20)
ca125[ctrl_idx] *= hemo

# Mild age trend in healthy (older → slightly lower median)
age_ctrl = age[ctrl_idx]
age_factor = np.clip(1.0 + (-0.002)*(age_ctrl - 45), 0.85, 1.15)
ca125[ctrl_idx] *= age_factor

# Ensure small false-positive tail (~1–3%) >35 U/mL in healthy
healthy_vals = ca125[ctrl_idx]
prop_healthy_high = (healthy_vals > 35).mean() if healthy_vals.size else 0.0
if prop_healthy_high < 0.01:
    bump_n = max(5, int(0.012 * ctrl_ids.size))
    if bump_n > 0 and ctrl_ids.size > 0:
        sel = rng.choice(ctrl_ids, size=min(bump_n, ctrl_ids.size), replace=False)
        ca125[sel] = np.maximum(ca125[sel], rng.uniform(36, 60, size=sel.size))
elif prop_healthy_high > 0.03:
    # If too high, softly pull some down near 30–34
    high_idx = np.where(ctrl_idx & (ca125 > 35))[0]
    pull_n = int(0.5 * high_idx.size)
    if pull_n > 0:
        sel = rng.choice(high_idx, size=pull_n, replace=False)
        ca125[sel] = rng.uniform(20, 34, size=sel.size)

# Cases baseline: median ~80 U/mL with heavy right tail; positive size effect; NO hemodilution
case_base = rtrunclognorm(median=80.0, sigma=0.80, low=2, high=2000, size=case_ids.size)

# Size effect: modest exponential in (size - 9 cm)
beta_size = 0.03
size_eff = np.exp(beta_size * (tumor_size_cm[case_ids] - 9.0))
ca125[case_ids] = np.clip(case_base * size_eff, 2, 2000)

# Ensure ≥35% of cases >200 U/mL
case_vals = ca125[case_ids]
prop_case_200 = (case_vals > 200).mean() if case_vals.size else 0.0
if prop_case_200 < 0.35:
    need = int(0.35 * case_ids.size - (case_vals > 200).sum())
    if need > 0:
        bump_candidates = case_ids[ca125[case_ids] <= 200]
        if bump_candidates.size > 0:
            sel = rng.choice(bump_candidates, size=min(need, bump_candidates.size), replace=False)
            ca125[sel] = rng.uniform(210, 600, size=sel.size)

# -----------------------------
# 5) Ultrasound risk: primarily size-driven + moderate CA-125 correlation (ρ≈0.3–0.5)
# -----------------------------
eps = 1e-6
log_ca = np.log(ca125/35.0 + eps)

# Tuned weights: size dominates; CA-125 contributes; intercept calibrated roughly by hand
alpha, gamma_size, gamma_ca = -4.2, 0.28, 0.55
risk_linear = alpha + gamma_size * tumor_size_cm + gamma_ca * log_ca
ultrasound_risk_score = sigmoid(risk_linear)

# Plausibility constraints
# - Controls with no mass: keep ≤0.08
mask_no_mass = ctrl_idx & (tumor_size_cm == 0)
ultrasound_risk_score[mask_no_mass] = np.minimum(ultrasound_risk_score[mask_no_mass], 0.08)

# - Very high CA-125 but spuriously low risk in cancers → gently bump
mask_lowrisk_highca = case_idx & (ultrasound_risk_score < 0.10) & (ca125 > 200)
ultrasound_risk_score[mask_lowrisk_highca] = 0.25

# Keep within [0, 0.99] to leave headroom
ultrasound_risk_score = np.clip(ultrasound_risk_score, 0.0, 0.99)

# -----------------------------
# 6) BRCA+ cases skew younger; adjust case BRCA prevalence to ~15% if needed
# -----------------------------
# (a) Post-hoc adjust case BRCA prevalence to target 15% within [12%, 18%]
case_rate = brca_status[case_idx].mean() if case_idx.any() else 0.0
target_brca = 0.15
low, high = 0.12, 0.18

if case_idx.any() and not (low <= case_rate <= high):
    case_pos = np.where(case_idx & (brca_status == 1))[0]
    case_neg = np.where(case_idx & (brca_status == 0))[0]
    current_pos = case_pos.size
    target_pos = int(round(target_brca * case_idx.sum()))
    if target_pos > current_pos and case_neg.size > 0:
        to_flip = min(target_pos - current_pos, case_neg.size)
        flip = rng.choice(case_neg, size=to_flip, replace=False)
        brca_status[flip] = 1
    elif target_pos < current_pos and case_pos.size > 0:
        to_flip = min(current_pos - target_pos, case_pos.size)
        flip = rng.choice(case_pos, size=to_flip, replace=False)
        brca_status[flip] = 0

# (b) Shift ages down by ~6–10y for BRCA+ cases; resample menopause for those
brca_case = (brca_status == 1) & case_idx
n_shift = brca_case.sum()
if n_shift > 0:
    age[brca_case] = np.clip(age[brca_case] - rng.integers(6, 11, size=n_shift), 18, 90)
    # resample menopause consistently with new age
    p_post2 = sigmoid((age - 51)/2.3)
    menopausal_status[brca_case] = (rng.random(n_shift) < p_post2[brca_case]).astype(int)

# -----------------------------
# 7) Assemble dataframe
# -----------------------------
df = pd.DataFrame({
    "age": age.astype(int),
    "menopausal_status": menopausal_status.astype(int),
    "bmi": np.round(bmi, 2),
    "parity": parity.astype(int),
    "family_history": family_history.astype(int),
    "mht_use": mht_use.astype(int),
    "brca_status": brca_status.astype(int),
    "ca125": np.round(ca125, 1),
    "ultrasound_risk_score": np.round(ultrasound_risk_score, 3),
    "tumor_size_cm": np.round(tumor_size_cm, 2),
    "diagnosis_label": diagnosis.astype(int),
})

# -----------------------------
# 8) Quick QA summary (sanity checks against table)
# -----------------------------
def safe_mean(x): 
    return float(np.mean(x)) if len(x) else float('nan')

def corr(a, b):
    if len(a) < 3: return np.nan
    a = a - a.mean(); b = b - b.mean()
    denom = np.sqrt((a*a).sum() * (b*b).sum())
    return float((a*b).sum()/denom) if denom > 0 else np.nan

ctrl = df.diagnosis_label == 0
case = df.diagnosis_label == 1

summary = {
    "n_rows": int(len(df)),
    "pct_cancer": float(df['diagnosis_label'].mean()),
    "age_mean_ctrl": safe_mean(df.loc[ctrl, 'age']),
    "age_mean_case": safe_mean(df.loc[case, 'age']),
    "bmi_mean_ctrl": safe_mean(df.loc[ctrl, 'bmi']),
    "bmi_mean_case": safe_mean(df.loc[case, 'bmi']),
    "fh_rate_ctrl": float(df.loc[ctrl, 'family_history'].mean()) if ctrl.any() else np.nan,
    "fh_rate_case": float(df.loc[case, 'family_history'].mean()) if case.any() else np.nan,
    "mht_rate_post_ctrl": float(df.loc[ctrl & (df.menopausal_status==1), 'mht_use'].mean()) if (ctrl & (df.menopausal_status==1)).any() else np.nan,
    "mht_rate_post_case": float(df.loc[case & (df.menopausal_status==1), 'mht_use'].mean()) if (case & (df.menopausal_status==1)).any() else np.nan,
    "brca_rate_among_cases": float(df.loc[case, 'brca_status'].mean()) if case.any() else np.nan,
    "overall_brca_rate": float(df['brca_status'].mean()),
    "healthy_ca125_over_35_pct": float((df.loc[ctrl, 'ca125'] > 35).mean()) if ctrl.any() else np.nan,
    "case_ca125_over_200_pct": float((df.loc[case, 'ca125'] > 200).mean()) if case.any() else np.nan,
    "median_size_cases_cm": float(df.loc[case, 'tumor_size_cm'].median()) if case.any() else np.nan,
    "median_size_controls_cm_nonzero": float(df.loc[ctrl & (df.tumor_size_cm > 0), 'tumor_size_cm'].median()) if (ctrl & (df.tumor_size_cm > 0)).any() else np.nan,
    "ultrasound_risk_mean_cases": safe_mean(df.loc[case, 'ultrasound_risk_score']),
    "ultrasound_risk_mean_controls": safe_mean(df.loc[ctrl, 'ultrasound_risk_score']),
    "corr_risk_ca125_cases": corr(df.loc[case, 'ultrasound_risk_score'].values, df.loc[case, 'ca125'].values),
}

# -----------------------------
# 9) Save
# -----------------------------
csv_path = "synthetic_clinical_dataset.csv"
df.to_csv(csv_path, index=False)

# For notebook users: display head & summary
df.head(10), summary


(   age  menopausal_status    bmi  parity  family_history  mht_use  \
 0   53                  1  19.94       5               0        0   
 1   37                  0  22.80       0               0        0   
 2   59                  1  26.16       3               0        0   
 3   61                  1  23.69       1               0        1   
 4   26                  0  38.53       1               0        0   
 5   34                  0  26.34       3               0        0   
 6   51                  1  34.68       3               0        0   
 7   46                  0  25.80       2               0        0   
 8   49                  0  28.12       2               0        0   
 9   39                  0  20.06       0               1        0   
 
    brca_status  ca125  ultrasound_risk_score  tumor_size_cm  diagnosis_label  
 0            0   12.9                  0.009           0.00                0  
 1            0   34.6                  0.015           0.00        